In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings 
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *
from statsforecast import StatsForecast
from statsforecast.models import (
    Naive,WindowAverage, ARIMA, 
    AutoARIMA,SeasonalNaive,HoltWinters,
    CrostonClassic as Croston, HistoricAverage,DynamicOptimizedTheta as DOT,
    SeasonalNaive
)
from dataclasses import dataclass
from typing import Optional, List
from itertools import product

warnings.filterwarnings("ignore")  # To ignore warnings from pandas/numpy

In [ ]:
user="Lilian"

In [ ]:
#New data class created to handle configuration parameters
@dataclass
class ForecastConfig:
    
    # Forecast parameters
    h: int = 8                          
    season_length: int = 4              
    
    # Cross-validation parameters
    n_windows: int = 2                  
    step_size: Optional[int] = None     
    
    # Train-test split parameters
    train_size: Optional[int] = None    # Use all available data except test
    test_size: Optional[int] = None     # Auto-set to h in __post_init__
    
    # Plotting parameters
    n_samples: int = 4                  # Plot 4 random samples
    models_to_plot: Optional[List[str]] = None  # 
    
    # Other settings
    confidence_level: int = 95          # 95% confidence intervals
    n_jobs: int = -1                    
    
    def __post_init__(self):
        
        if self.step_size is None:
            self.step_size = self.h
        if self.test_size is None:
            self.test_size = self.h
        if self.models_to_plot is None:
            self.models_to_plot = ['Naive', 'ARIMA_manual', 'SARIMA']

In [ ]:
def get_models(config):
    
    models = [Naive(), HistoricAverage(), WindowAverage(window_size=4),
        SeasonalNaive(season_length=4), ARIMA(order=(1, 1, 1), alias="ARIMA_manual"),
        AutoARIMA(seasonal=True, season_length=4, alias="SARIMA"),
    ]
    return models

def evaluate_train_test(df, target_name, config):

    if config.train_size is None:
        train = df.groupby('unique_id').apply(
            lambda x: x.iloc[:-config.test_size]
        ).reset_index(drop=True)
    else:
        train = df.groupby('unique_id').apply(
            lambda x: x.iloc[-(config.train_size + config.test_size):-config.test_size]
        ).reset_index(drop=True)
    
    test = df.groupby('unique_id').apply(
        lambda x: x.iloc[-config.test_size:]
    ).reset_index(drop=True)
    
    sf = StatsForecast(
        models=get_models(config),
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )
    sf.fit(df=train)
    preds = sf.predict(h=config.h)

    preds_df = pd.merge(test, preds.reset_index(), on=['ds', 'unique_id'], how='left')
    models = [col for col in preds.columns if col not in ['unique_id', 'ds']]
    
    eval_df = evaluate(preds_df, metrics=[mae, mse, rmse], models=models)
    
    mae_df = eval_df[eval_df['metric'] == 'mae'].copy()
    mae_df['best_model'] = mae_df[models].idxmin(axis=1)
    
    print(f"\n📈 Best Models (Train-Test Split - based on MAE):")
    print(mae_df['best_model'].value_counts())
    
    return eval_df, preds_df, train, test 

def evaluate_model_cross(df, target_name, config):
    
    sf = StatsForecast(
        models=get_models(config),
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )

    print(f"   Running cross-validation...")
    cv_df = sf.cross_validation(
        df=df,
        h=config.h,
        n_windows=config.n_windows,
        step_size=config.step_size
    )
    
    # Define model columns
    exclude_cols = ['unique_id', 'ds', 'y', 'cutoff', 'metric']
    model_cols = [col for col in cv_df.columns if col not in exclude_cols]

    # Evaluate per cutoff window
    all_results = []
    
    for cutoff in cv_df['cutoff'].unique():
        cutoff_data = cv_df[cv_df['cutoff'] == cutoff]
        
        # Evaluate all metrics for this cutoff
        cutoff_eval = evaluate(cutoff_data, metrics=[mae, mse, rmse], models=model_cols)
        cutoff_eval['cutoff'] = cutoff
        
        # Add best_model column (lowest value per row, regardless of metric)
        cutoff_eval['best_model'] = cutoff_eval[model_cols].idxmin(axis=1)
        cutoff_eval['best_value'] = cutoff_eval[model_cols].min(axis=1)
        
        all_results.append(cutoff_eval)
        
        # Print MAE summary for this cutoff
        cutoff_mae = cutoff_eval[cutoff_eval['metric'] == 'mae']
        print(f"\n   Cutoff {cutoff.strftime('%Y-%m-%d')} (MAE best models):")
        print(f"   {cutoff_mae['best_model'].value_counts().to_dict()}")

    # Combine all cutoff results into single dataframe
    eval_df = pd.concat(all_results, ignore_index=True)
    
    # Print overall summary
    mae_overall = eval_df[eval_df['metric'] == 'mae']
    print(f"\n📈 Overall Best Models (all cutoffs - based on MAE):")
    print(mae_overall['best_model'].value_counts())
    
    return eval_df, cv_df

def sensitivity_analysis_with_production_forecasts(df, target_name, param_grid=None, production_horizons=[4, 8], save_path=None):
    
    # Default parameter grid if not provided
    if param_grid is None:
        param_grid = {
            'h': [4, 8],
            'test_size': [4, 8],
            'train_size': [None, 20, 28],
            'n_windows': [2, 3, 4],
            'step_size': [None, 2, 4],
        }
   
    print("\nPHASE 1: SENSITIVITY ANALYSIS")
    
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    combinations = list(product(*param_values))
    
    print(f"Testing {len(combinations)} configurations...")
    
    all_results = []
    
    for i, combo in enumerate(combinations):
        params = dict(zip(param_names, combo))
        
        # Skip invalid combinations
        if params.get('test_size') and params.get('h'):
            if params['test_size'] < params['h']:
                continue
        
        print(f"\nConfiguration {i+1}/{len(combinations)}: {params}")
        
        try:
            config = ForecastConfig(
                h=params.get('h', 8),
                train_size=params.get('train_size'),
                test_size=params.get('test_size'),
                n_windows=params.get('n_windows', 2),
                step_size=params.get('step_size'),
                season_length=4,
                n_samples=4,
                confidence_level=95,
                n_jobs=-1
            )
            
            # Run train-test evaluation
            eval_traintest, preds_traintest, train_df, test_df = evaluate_train_test(
                df, target_name, config
            )
            
            # Run cross-validation
            eval_cv, cv_df = evaluate_model_cross(df, target_name, config)
            
            # Extract MAE results
            mae_cv = eval_cv[eval_cv['metric'] == 'mae'].copy()
            mae_traintest = eval_traintest[eval_traintest['metric'] == 'mae'].copy()
            
            # Get model columns
            exclude_cols = ['unique_id', 'ds', 'y', 'cutoff', 'metric', 'best_model', 'best_value']
            model_cols = [col for col in mae_cv.columns if col not in exclude_cols]
            
            # Add best_model to train-test if not present
            if 'best_model' not in mae_traintest.columns:
                mae_traintest['best_model'] = mae_traintest[model_cols].idxmin(axis=1)
            
            # Store results
            for uid in df['unique_id'].unique():
                uid_cv = mae_cv[mae_cv['unique_id'] == uid]
                uid_traintest = mae_traintest[mae_traintest['unique_id'] == uid]
                
                cv_best_model = uid_cv['best_model'].mode().iloc[0] if len(uid_cv) > 0 else None
                cv_best_count = (uid_cv['best_model'] == cv_best_model).sum()
                cv_total = len(uid_cv)
                cv_consistency = cv_best_count / cv_total if cv_total > 0 else 0
                
                traintest_best_model = uid_traintest['best_model'].iloc[0] if len(uid_traintest) > 0 else None
                
                result_row = {
                    'unique_id': uid,
                    'config_id': i + 1,
                    **params,
                    'cv_best_model': cv_best_model,
                    'cv_consistency': cv_consistency,
                    'traintest_best_model': traintest_best_model,
                    'cv_traintest_agree': cv_best_model == traintest_best_model,
                }
                
                # Add MAE values for each model
                for model in model_cols:
                    model_mae = uid_cv[model].mean() if len(uid_cv) > 0 else None
                    result_row[f'{model}_mae'] = model_mae
                
                all_results.append(result_row)
            
            print(f"   ✓ Completed successfully")
            
        except Exception as e:
            print(f"   ✗ Error: {str(e)}")
            continue
    
    results_df = pd.DataFrame(all_results)
    
    print("\nPHASE 2: IDENTIFYING BEST MODELS PER HORIZON")
    recommendations_per_horizon = {}
    
    for horizon in production_horizons:
        print(f"\n🎯 Analyzing h={horizon}...")
        
        # Filter results for this horizon
        horizon_results = results_df[results_df['h'] == horizon].copy()
        
        if len(horizon_results) == 0:
            print(f"   ⚠️ No results found for h={horizon}, skipping")
            continue
        
        # Get recommendations for each unique_id at this horizon
        horizon_recommendations = []
        
        for uid in horizon_results['unique_id'].unique():
            uid_data = horizon_results[horizon_results['unique_id'] == uid]
            
            # Most frequent CV best model
            cv_mode = uid_data['cv_best_model'].mode()
            cv_best = cv_mode.iloc[0] if len(cv_mode) > 0 else None
            cv_freq = (uid_data['cv_best_model'] == cv_best).sum() / len(uid_data)
            
            # Most frequent train-test best model
            tt_mode = uid_data['traintest_best_model'].mode()
            tt_best = tt_mode.iloc[0] if len(tt_mode) > 0 else None
            tt_freq = (uid_data['traintest_best_model'] == tt_best).sum() / len(uid_data)
            
            # Average consistency
            avg_consistency = uid_data['cv_consistency'].mean()
            
            # Determine recommendation
            if cv_best == tt_best and cv_freq >= 0.7 and avg_consistency >= 0.7:
                confidence = 'High'
                recommended_model = cv_best
                reason = "CV and Train-Test agree, high frequency and consistency"
            elif cv_best == tt_best and cv_freq >= 0.5:
                confidence = 'Medium-High'
                recommended_model = cv_best
                reason = "CV and Train-Test agree with moderate frequency"
            elif cv_freq >= 0.6:
                confidence = 'Medium'
                recommended_model = cv_best
                reason = f"CV favors {cv_best} ({cv_freq:.0%})"
            elif tt_freq >= 0.6:
                confidence = 'Medium'
                recommended_model = tt_best
                reason = f"Train-Test favors {tt_best} ({tt_freq:.0%})"
            else:
                confidence = 'Low'
                mae_cols = [col for col in uid_data.columns if col.endswith('_mae')]
                if mae_cols:
                    avg_maes = uid_data[mae_cols].mean()
                    recommended_model = avg_maes.idxmin().replace('_mae', '')
                else:
                    recommended_model = cv_best
                reason = "No clear winner - using lowest average MAE"
            
            # Get average MAE
            rec_mae_col = f'{recommended_model}_mae'
            avg_mae = uid_data[rec_mae_col].mean() if rec_mae_col in uid_data.columns else None
            
            recommendation = {
                'unique_id': uid,
                'horizon': horizon,
                'recommended_model': recommended_model,
                'confidence': confidence,
                'reason': reason,
                'cv_best_model': cv_best,
                'cv_frequency': cv_freq,
                'cv_consistency': avg_consistency,
                'traintest_best_model': tt_best,
                'avg_mae': avg_mae
            }
            
            horizon_recommendations.append(recommendation)
            mae_display= f"{avg_mae:,.0f}" if pd.notna(avg_mae)is not None else "N/A"
            print(f"   {uid}: {recommended_model} (confidence: {confidence}, MAE: {mae_display})")
            
            print(f"   {uid}: {recommended_model} (confidence: {confidence}, MAE: {avg_mae:,.0f})")
        
        recommendations_per_horizon[horizon] = pd.DataFrame(horizon_recommendations)
    

    print("\nPHASE 3: GENERATING PRODUCTION FORECASTS")
    
    production_forecasts = {}
    
    for horizon in production_horizons:
        if horizon not in recommendations_per_horizon:
            continue
            
        print(f"\n🔮 Generating forecasts for h={horizon}...")
        
        recommendations = recommendations_per_horizon[horizon]
        
        # Train on FULL dataset
        config_prod = ForecastConfig(
            h=horizon,
            season_length=4,
            confidence_level=95,
            n_jobs=-1
        )
        
        # Train all models on full data
        sf = StatsForecast(
            models=get_models(config_prod),
            freq='QS',
            n_jobs=-1,
            fallback_model=SeasonalNaive(season_length=4)
        )
        
        forecasts_df = sf.forecast(df=df, h=horizon, level=[95])
        
        # Create a "best model" forecast by selecting the recommended model for each unique_id
        best_forecasts = []
        
        for uid in df['unique_id'].unique():
            uid_forecasts = forecasts_df.reset_index()
            uid_forecasts = uid_forecasts[uid_forecasts['unique_id'] == uid]
            
            uid_rec = recommendations[recommendations['unique_id'] == uid]
            if len(uid_rec) == 0:
                print(f"   ⚠️ No recommendation found for {uid}, skipping")
                continue
            
            best_model = uid_rec['recommended_model'].iloc[0]
            
            for _, row in uid_forecasts.iterrows():
                best_row = {
                    'unique_id': uid,
                    'ds': row['ds'],
                    'recommended_model': best_model,
                    'forecast': row.get(best_model, np.nan),
                    'forecast_lo_95': row.get(f'{best_model}-lo-95', np.nan),
                    'forecast_hi_95': row.get(f'{best_model}-hi-95', np.nan)
                }
                best_forecasts.append(best_row)
        
        best_forecasts_df = pd.DataFrame(best_forecasts)
        
        production_forecasts[horizon] = {
            'all_models': forecasts_df,
            'best_model': best_forecasts_df,
            'recommendations': recommendations
        }
        
        print(f"   ✓ Generated {len(best_forecasts_df)} forecast periods")
        print(f"   ✓ Models used: {recommendations['recommended_model'].value_counts().to_dict()}")
    
        print("\nPHASE 4: SAVING RESULTS")
        
        if save_path:
            os.makedirs(save_path, exist_ok=True)
            rec_frames=list(recommendations_per_horizon.values())
            all_recs = pd.concat(rec_frames, ignore_index=True) if rec_frames else pd.DataFrame()
            # Save sensitivity analysis results
            sensitivity_file = os.path.join(save_path, f'{target_name.lower().replace(" ", "_")}_sensitivity_analysis.csv')
            with pd.ExcelWriter(sensitivity_file, engine='openpyxl') as writer:
                results_df.to_excel(writer, sheet_name='All_Configurations', index=False)
                all_recs.to_excel(writer, sheet_name='Recommendations', index=False)
            
            print(f"✓ Sensitivity analysis saved: {sensitivity_file}")
            
            # Save production forecasts for each horizon
            for horizon, forecasts in production_forecasts.items():
                horizon_file = os.path.join(save_path, f'{target_name.lower().replace(" ", "_")}_production_h{horizon}.xlsx')
                with pd.ExcelWriter(horizon_file, engine='openpyxl') as writer:
                    forecasts['all_models'].reset_index().to_excel(writer, sheet_name='All_Models', index=False)
                    forecasts['best_model'].to_excel(writer, sheet_name='Best_Model_Forecast', index=False)
                    forecasts['recommendations'].to_excel(writer, sheet_name='Model_Selection', index=False)
                print(f"✓ Production forecasts (h={horizon}) saved: {horizon_file}")
        else:
            print("   ⚠️ No save_path provided - skipping file export")
                
        
    return {
        'sensitivity_results': results_df,
        'production_forecasts': production_forecasts,
        'recommendations_per_horizon': recommendations_per_horizon
    }

### Population Forecast

In [ ]:
def load_data(filepath, states=None):
    
    df=pd.read_csv(filepath)
    #Filtering for certain years
    df = df[(df['Period'] >= '2017Q1') & (df['Period'] <= '2024Q4')].copy()
    if states is not None:
        if isinstance(states, str):
            states = [states]
        print(f"Filtering data for states: {states}")
        df = df[df['State'].isin(states)].copy()
        print(f"Filtered to {len(df)} rows across {df['State'].unique()} state(s)")
        if len(df) == 0:
            raise ValueError("No data available after filtering by states.")    
    
    df['unique_id']=df['State']
    df['ds']=pd.to_datetime(df['Period'])
    df=df.sort_values(['unique_id','ds']).reset_index(drop=True)
    df_pop=df[['unique_id','ds','Population']].copy()
    df_pop.columns = ['unique_id', 'ds', 'y']
    

    return df_pop,df

pop_csv_path=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\Prebuilt_panels\medi_pop.csv"
pop_save_path=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\Forecast\Pop\\"

In [ ]:
df_pop, df_original = load_data(filepath=pop_csv_path, states=['IN','MI','OH','IL'])
config = ForecastConfig()

# This is the manual way using config if you want to run individual evaluations
print("\n1. Train-Test Evaluation...")
eval_traintest, preds_traintest, train, test = evaluate_train_test(
    df_pop, "Population", config
)

print("\n2. Cross-Validation...")
eval_cv, cv_df = evaluate_model_cross(
    df_pop, "Population", config
)

print("\n3. Generate Forecasts...")
sf = StatsForecast(
    models=get_models(config),
    freq='QS',
    n_jobs=config.n_jobs,
    fallback_model=SeasonalNaive(season_length=config.season_length)
)
forecasts = sf.forecast(df=df_pop, h=config.h, level=[config.confidence_level])

# Save
forecasts.reset_index().to_csv(
    os.path.join(pop_save_path, f'population_forecast_h{config.h}.csv'),
    index=False
)


In [ ]:

#Sensitivity analysis with production forecasts
df_pop, df_original = load_data(filepath=pop_csv_path, states=['IN','MI','OH','IL'])
param_grid = {
    'h': [4, 8],
    'test_size': [4, 8],
    'train_size': [None, 26],
    'n_windows': [2, 3],
    'step_size': [4],
}
results = sensitivity_analysis_with_production_forecasts(
    df=df_pop,
    target_name="Population",
    param_grid=param_grid,
    production_horizons=[4, 8],
    save_path=pop_save_path
)

### SDUD with pop - Statistical Models

In [ ]:
pop_path = rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\Prebuilt_panels\medi_pop_1.csv"
drug_path = rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\Prebuilt_panels\P1_nopop.csv"

df_pop = pd.read_csv(pop_path)
df_drug = pd.read_csv(drug_path)
df_pop = df_pop[df_pop['Period'] >= '2017Q1'].copy()

print("medi_pop columns:", df_pop.columns.tolist())
print("P1_nopop columns:", df_drug.columns.tolist())

df_merged = pd.merge(df_drug, df_pop, on=['State', 'Period'], how='left')

existing_periods = df_drug['Period'].unique()
all_pop_periods = df_pop['Period'].unique()
future_periods = [p for p in all_pop_periods if p not in existing_periods]

print(f"\nFuture periods to expand: {future_periods}")
state_atc_combos = df_drug[['State', 'ATC2 Class']].drop_duplicates()
df_pop_future = df_pop[df_pop['Period'].isin(future_periods)].copy()
future_rows = []

for _, pop_row in df_pop_future.iterrows():
    state = pop_row['State']
    # Get all ATC2 classes for this state
    atc_classes = state_atc_combos[state_atc_combos['State'] == state]['ATC2 Class'].unique()
    
    for atc_class in atc_classes:
        new_row = {
            'State': state,
            'ATC2 Class': atc_class,
            'Period': pop_row['Period'],
            'Population': pop_row.get('Population', np.nan),
            'Forecast_low_95': pop_row.get('Forecast_low_95', np.nan),
            'Forecast_high_95': pop_row.get('Forecast_high_95', np.nan),
            'Units Reimbursed': np.nan,  # No drug data for future periods
            'Number of Prescriptions': np.nan
        }
        if 'Year' in pop_row:
            new_row['Year'] = pop_row['Year']
        if 'Quarter' in pop_row:
            new_row['Quarter'] = pop_row['Quarter']
        
        future_rows.append(new_row)

df_future = pd.DataFrame(future_rows)
print(f"Created {len(df_future)} future period rows")
df_final = pd.concat([df_merged, df_future], ignore_index=True)

columns = ['State', 'ATC2 Class', 'Year', 'Quarter', 'Period', 
           'Units Reimbursed', 'Number of Prescriptions', 
           'Population', 'Forecast_low_95', 'Forecast_high_95']
available_columns = [col for col in columns if col in df_final.columns]

df_final = df_final[available_columns].copy()
df_final = df_final.sort_values(['State', 'ATC2 Class', 'Period']).reset_index(drop=True)
df_final['Period']=pd.to_datetime(df_final['Period'])

print(f"\nFinal DataFrame shape: {df_final.shape}")
print(f"Period range: {df_final['Period'].min()} to {df_final['Period'].max()}")
print(f"Datatypes:\n{df_final.dtypes}")

output_path = rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\Prebuilt_panels\P1_withpop.csv"
df_final.to_csv(output_path, index=False)
print(f"\nMerged DataFrame saved to: {output_path}")

In [ ]:
#Defining functions used above but this time for exogenous feature (pop)
def load_data_exog(filepath, states=None):

    df = pd.read_csv(filepath)
    df['ds'] = pd.to_datetime(df['Period'], errors='coerce')
    
    df = df[(df['ds'] >= '2017-01-01') & (df['ds'] <= '2024-10-01')].copy()
    
    if states is not None:
        if isinstance(states, str):
            states = [states]
        print(f"Filtering data for states: {states}")
        df = df[df['State'].isin(states)].copy()
        print(f"Filtered to {len(df)} rows across {df['State'].unique()} state(s)")
        if len(df) == 0:
            raise ValueError("No data available after filtering by states.")
    
    df['unique_id'] = df['State'] + '_' + df['ATC2 Class']
    df = df.sort_values(['unique_id','ds']).reset_index(drop=True)

    # ⭐ DON'T include population - we'll get it from scenarios
    df_units = df[['unique_id', 'ds', 'Units Reimbursed']].copy()
    df_units.columns = ['unique_id', 'ds', 'y']
    df_units = df_units.dropna(subset=['y'])  # ← Only drop NaN in y

    df_prescriptions = df[['unique_id', 'ds', 'Number of Prescriptions']].copy()
    df_prescriptions.columns = ['unique_id', 'ds', 'y']
    df_prescriptions = df_prescriptions.dropna(subset=['y'])  # ← Only drop NaN in y

    return df_units, df_prescriptions, df

def pop_scenarios_exog(filepath,states=None):

    df = pd.read_csv(filepath)
    df['ds'] = pd.to_datetime(df['Period'], errors='coerce')
    
    if states is not None:
        if isinstance(states, str):
            states = [states]
        df = df[df['State'].isin(states)].copy()
    
    df['unique_id'] = df['State'] + '_' + df['ATC2 Class']
    df = df.sort_values(['unique_id', 'ds']).reset_index(drop=True)
    
    df_historical = df[df['ds'] <= '2024-10-01'].copy()
    df_future = df[df['ds'] > '2024-10-01'].copy()

    pop_historical = df_historical[['unique_id', 'ds', 'population']].copy()
    pop_historical.columns = ['unique_id', 'ds', 'population']

    scenarios={}
    pop_future_point = df_future[['unique_id', 'ds', 'population']].copy()
    pop_future_point.columns = ['unique_id', 'ds', 'population']
    scenarios['point'] = pd.concat([pop_historical, pop_future_point], ignore_index=True)
    
    # Scenario 2: Lower bound (uses Forecast_low_95 column)
    pop_future_low = df_future[['unique_id', 'ds', 'Forecast_low_95']].copy()
    pop_future_low.columns = ['unique_id', 'ds', 'population']
    scenarios['low_95'] = pd.concat([pop_historical, pop_future_low], ignore_index=True)
    
    # Scenario 3: Upper bound (uses Forecast_high_95 column)
    pop_future_high = df_future[['unique_id', 'ds', 'Forecast_high_95']].copy()
    pop_future_high.columns = ['unique_id', 'ds', 'population']
    scenarios['high_95'] = pd.concat([pop_historical, pop_future_high], ignore_index=True)
    
    # Verify we have future data
    for scenario_name, scenario_df in scenarios.items():
        future_count = len(scenario_df[scenario_df['ds'] > '2024-10-01'])
        print(f"   {scenario_name}: {future_count} future population records")
    
    return scenarios

def get_models_exog(config, use_exog=True):
    
    if use_exog:
        models = [
            # Baseline models (ignore exog)
            Naive(),
            SeasonalNaive(season_length=config.season_length),
            HistoricAverage(),
            WindowAverage(window_size=4),
            
            # Models with exogenous support
            AutoARIMA(
                seasonal=True, 
                season_length=config.season_length, 
                alias="SARIMAX"
            ),
            AutoARIMA(
                seasonal=False,
                season_length=config.season_length,
                alias="ARIMAX"
            ),
            # Note: Can add more exog-aware models here
        ]
    else:
        # Standard models without exog
        models = [
            Naive(),
            HistoricAverage(),
            WindowAverage(window_size=4),
            SeasonalNaive(season_length=config.season_length),
            ARIMA(order=(1, 1, 1), alias="ARIMA_manual"),
            AutoARIMA(seasonal=True, season_length=config.season_length, alias="SARIMA"),
        ]
    
    return models

def evaluate_train_test_exog(df,target_name,config,population_df=None):
    
    # Split train/test
    if config.train_size is None:
        train = df.groupby('unique_id').apply(
            lambda x: x.iloc[:-config.test_size]
        ).reset_index(drop=True)
    else:
        train = df.groupby('unique_id').apply(
            lambda x: x.iloc[-(config.train_size + config.test_size):-config.test_size]
        ).reset_index(drop=True)
    
    test = df.groupby('unique_id').apply(
        lambda x: x.iloc[-config.test_size:]
    ).reset_index(drop=True)
    
    # Get models
    use_exog = population_df is not None
    models = get_models_exog(config, use_exog=use_exog)
    
    # Initialize StatsForecast
    sf = StatsForecast(
        models=models,
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )
    
    # Fit and predict with/without exogenous variables
    if use_exog:

        if 'population' in train.columns:
            train = train.drop(columns=['population'])
        if 'population' in test.columns:
            test = test.drop(columns=['population'])

        # Prepare exogenous data for train and test
        pop_train = population_df[population_df['ds'].isin(train['ds'])].copy()
        pop_test = population_df[population_df['ds'].isin(test['ds'])].copy()
        
        # Merge population with train data
        train_with_pop = train.merge(pop_train, on=['unique_id', 'ds'], how='left')
        
        # Fit with exogenous
        sf.fit(df=train_with_pop[['unique_id', 'ds', 'y', 'population']])
        
        # Predict with future exogenous
        test_with_pop = test.merge(pop_test, on=['unique_id', 'ds'], how='left')
        preds = sf.predict(h=config.h, X_df=test_with_pop[['unique_id', 'ds', 'population']])
    else:
        sf.fit(df=train)
        preds = sf.predict(h=config.h)
    
    # Merge predictions with actuals
    preds_df = pd.merge(test, preds.reset_index(), on=['ds', 'unique_id'], how='left')
    
    # Evaluate
    model_cols = [col for col in preds.columns if col not in ['unique_id', 'ds']]
    eval_df = evaluate(preds_df, metrics=[mae, mse, rmse], models=model_cols)
    
    # Add best model
    mae_df = eval_df[eval_df['metric'] == 'mae'].copy()
    mae_df['best_model'] = mae_df[model_cols].idxmin(axis=1)
    
    exog_label = " (with population)" if use_exog else " (no exog)"
    print(f"\n📈 Best Models (Train-Test Split - {target_name}{exog_label}):")
    print(mae_df['best_model'].value_counts())
    
    return eval_df, preds_df, train, test

def CV_with_exog(df,target_name,config,population_df=None):
    
    use_exog = population_df is not None
    models = get_models_exog(config, use_exog=use_exog)
    
    sf = StatsForecast(
        models=models,
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )
    
    print(f"   Running cross-validation...")
    
    # Cross-validation with/without exogenous
    if use_exog:

        df_for_cv = df.copy()
        if 'population' in df_for_cv.columns:
            df_for_cv = df_for_cv.drop(columns=['population'])
            
        df_with_pop = df.merge(population_df, on=['unique_id', 'ds'], how='left')
        cv_df = sf.cross_validation(
            df=df_with_pop[['unique_id', 'ds', 'y', 'population']],
            h=config.h,
            n_windows=config.n_windows,
            step_size=config.step_size
        )
    else:
        cv_df = sf.cross_validation(
            df=df,
            h=config.h,
            n_windows=config.n_windows,
            step_size=config.step_size
        )
    
    # Evaluate per cutoff
    exclude_cols = ['unique_id', 'ds', 'y', 'cutoff', 'metric', 'population']
    model_cols = [col for col in cv_df.columns if col not in exclude_cols]
    
    all_results = []
    
    for cutoff in cv_df['cutoff'].unique():
        cutoff_data = cv_df[cv_df['cutoff'] == cutoff]
        
        cutoff_eval = evaluate(cutoff_data, metrics=[mae, mse, rmse], models=model_cols)
        cutoff_eval['cutoff'] = cutoff
        cutoff_eval['best_model'] = cutoff_eval[model_cols].idxmin(axis=1)
        cutoff_eval['best_value'] = cutoff_eval[model_cols].min(axis=1)
        
        all_results.append(cutoff_eval)
        
        cutoff_mae = cutoff_eval[cutoff_eval['metric'] == 'mae']
        print(f"\n   Cutoff {cutoff.strftime('%Y-%m-%d')} (MAE best models):")
        print(f"   {cutoff_mae['best_model'].value_counts().to_dict()}")
    
    eval_df = pd.concat(all_results, ignore_index=True)
    
    mae_overall = eval_df[eval_df['metric'] == 'mae']
    exog_label = " (with population)" if use_exog else ""
    print(f"\n📈 Overall Best Models{exog_label}:")
    print(mae_overall['best_model'].value_counts())
    
    return eval_df, cv_df

def generate_future_forecasts(df_historical, target_name, config, population_scenario, horizon=None,save_path=None):
    
    if horizon is None:
        horizon = config.h
    use_exog = population_scenario is not None
    models = get_models_exog(config, use_exog=use_exog)
    
    sf = StatsForecast(
        models=models,
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )
    
    # Train on full historical data
    if use_exog:
        # Get historical population
        pop_historical = population_scenario[
            population_scenario['ds'] <= df_historical['ds'].max()
        ].copy()
        
        df_train = df_historical.merge(pop_historical, on=['unique_id', 'ds'], how='left')
        sf.fit(df=df_train[['unique_id', 'ds', 'y', 'population']])
        
        # Get future population for forecasting
        last_date = df_historical['ds'].max()
        future_dates = pd.date_range(
            start=last_date + pd.DateOffset(months=3),
            periods=horizon,
            freq='QS'
        )
        
        # Create future exogenous dataframe
        future_exog_list = []
        for uid in df_historical['unique_id'].unique():
            for date in future_dates:
                pop_val = population_scenario[
                    (population_scenario['unique_id'] == uid) & 
                    (population_scenario['ds'] == date)
                ]['population'].values
                
                if len(pop_val) > 0:
                    future_exog_list.append({
                        'unique_id': uid,
                        'ds': date,
                        'population': pop_val[0]
                    })
        
        future_exog_df = pd.DataFrame(future_exog_list)
        forecasts_df = sf.predict(h=horizon, X_df=future_exog_df)
    else:
        sf.fit(df=df_historical,)
        forecasts_df = sf.predict(h=horizon)
    
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        filename = os.path.join(
            save_path, 
            f'{target_name.lower().replace(" ", "_")}_future_h{horizon}.csv'
        )
        forecasts_df.reset_index().to_csv(filename, index=False)
        print(f"✅ Saved: {filename}")
    
    return forecasts_df

def forecast_with_population_scenarios(filepath, target_col, states=None, config=None, horizons=None, save_path=None):
    
    if config is None:
        config = ForecastConfig()
    
    # ⭐ If horizons not specified, use config.h
    if horizons is None:
        horizons = [config.h]
    
    print("\n" + "="*70)
    print(f"FORECASTING: {target_col}")
    print(f"Horizons: {horizons} quarters")
    print("="*70)
    
    # Load data
    print("\n1. Loading data...")
    df_units, df_prescriptions, df_full = load_data_exog(filepath, states)
    
    df = df_units if target_col == 'Units Reimbursed' else df_prescriptions
    target_name = target_col
    
    print(f"   • Unique series: {df['unique_id'].nunique()}")
    print(f"   • Date range: {df['ds'].min()} to {df['ds'].max()}")
    
    # Create population scenarios
    print("\n2. Creating population scenarios...")
    pop_scenarios = pop_scenarios_exog(filepath, states)
    
    results = {
        'point': {},
        'low_95': {},
        'high_95': {}
    }
    
    # Run for each scenario
    for scenario_name, pop_df in pop_scenarios.items():
        print(f"\n{'='*70}")
        print(f"SCENARIO: {scenario_name.upper()}")
        print("="*70)
        
        # Evaluation (uses config.h for train-test and CV)
        print("\n3. Train-Test Evaluation...")
        eval_tt, preds_tt, train, test = evaluate_train_test_exog(
            df, target_name, config, pop_df
        )
        
        print("\n4. Cross-Validation...")
        eval_cv, cv_df = CV_with_exog(
            df, target_name, config, pop_df
        )
        
        # Production forecasts for each horizon
        for h in horizons:
            print(f"\n5. Generating future forecasts (h={h})...")
            
            forecasts = generate_future_forecasts(
                df, target_name, config, pop_df, 
                horizon=h,  # ⭐ Override config.h with specific horizon
                save_path=os.path.join(save_path, scenario_name) if save_path else None
            )
            
            results[scenario_name][f'h{h}'] = {
                'forecasts': forecasts,
                'eval_train_test': eval_tt,
                'eval_cross_val': eval_cv,
                'predictions': preds_tt
            }
    
    print("\n" + "="*70)
    print("✅ COMPLETE!")
    print("="*70)
    
    return results                

In [ ]:
def sensitivity_analysis_with_exog(filepath, target_col, states=None, param_grid=None, production_horizons=[4, 8],
    population_scenario='point',  # 'point', 'low_95', or 'high_95'
    save_path=None):

    # Default parameter grid
    if param_grid is None:
        param_grid = {
            'h': [4, 8],
            'test_size': [4, 8],
            'train_size': [None, 20, 28],
            'n_windows': [2, 3, 4],
            'step_size': [4],
        }
    
    print("=" * 80); print(f"SENSITIVITY ANALYSIS WITH EXOGENOUS VARIABLES: {target_col}")
    print(f"Population Scenario: {population_scenario.upper()}"); print(f"Testing {len(list(product(*param_grid.values())))} configurations")
    print("=" * 80); print("\nPhase 1: Loading and preparing data...")
    
    df_units, df_prescriptions, df_full = load_data_exog(filepath, states)
    df = df_units if target_col == 'Units Reimbursed' else df_prescriptions
    target_name = target_col
    
    print(f"\nUnique series: {df['unique_id'].nunique()}")
    print(f"\nDate range: {df['ds'].min()} to {df['ds'].max()}")
    print(f"\nTotal observations: {len(df)}")
    
    # Get population scenarios
    pop_scenarios = pop_scenarios_exog(filepath, states)
    pop_df = pop_scenarios[population_scenario]
    
    print(f"\nPopulation scenario loaded: {population_scenario}")
    print(f"\nPopulation records: {len(pop_df)}")
    
    print("\nPhase 2: Running sensitivity analysis...")
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    combinations = list(product(*param_values))
    
    all_results = []
    all_errors = []  # Simplified error tracking - mean per config
    
    for i, combo in enumerate(combinations):
        params = dict(zip(param_names, combo))
        
        # Skip invalid combinations
        if params.get('test_size') and params.get('h'):
            if params['test_size'] < params['h']:
                continue
        
        if params['test_size'] != params['h']:
            continue

        print(f"\n{'─' * 80}"); print(f"Configuration {i+1}/{len(combinations)}: {params}"); print('─' * 80)
        
        try:
            config = ForecastConfig(
                h=params.get('h', 8),
                train_size=params.get('train_size'),
                test_size=params.get('test_size'),
                n_windows=params.get('n_windows', 2),
                step_size=params.get('step_size'),
                season_length=4,
                n_jobs=-1
            )
            
            # Train-Test Evaluation
            print("\nRunning train-test evaluation...")
            eval_tt, preds_tt, train_df, test_df = evaluate_train_test_exog(
                df, target_name, config, pop_df
            )
            
            # Cross-Validation
            print("\nRunning Cross-Validation...")
            eval_cv, cv_df = CV_with_exog(
                df, target_name, config, pop_df
            )
            
            exclude_cols = ['unique_id', 'ds', 'y', 'cutoff', 'metric', 'best_model', 'best_value', 'population']
            model_cols = [col for col in eval_cv.columns if col not in exclude_cols]
            
            # Extract MAE results
            mae_cv = eval_cv[eval_cv['metric'] == 'mae'].copy()
            mae_traintest = eval_tt[eval_tt['metric'] == 'mae'].copy()
            
            # Add best_model if not present
            if 'best_model' not in mae_traintest.columns:
                mae_traintest['best_model'] = mae_traintest[model_cols].idxmin(axis=1)
            
            # Collect Results Per Unique_ID 
            for uid in df['unique_id'].unique():
                uid_cv = mae_cv[mae_cv['unique_id'] == uid]
                uid_traintest = mae_traintest[mae_traintest['unique_id'] == uid]
                
                # CV analysis
                cv_best_model = uid_cv['best_model'].mode().iloc[0] if len(uid_cv) > 0 else None
                cv_best_count = (uid_cv['best_model'] == cv_best_model).sum()
                cv_total = len(uid_cv)
                cv_consistency = cv_best_count / cv_total if cv_total > 0 else 0
                
                # Train-test analysis
                traintest_best_model = uid_traintest['best_model'].iloc[0] if len(uid_traintest) > 0 else None
                
                # Build result row
                result_row = {
                    'unique_id': uid,
                    'config_id': i + 1,
                    'population_scenario': population_scenario,
                    **params,
                    'cv_best_model': cv_best_model,
                    'cv_consistency': cv_consistency,
                    'traintest_best_model': traintest_best_model,
                    'cv_traintest_agree': cv_best_model == traintest_best_model,
                }
                
                # Add MAE values for each model
                for model in model_cols:
                    cv_mae = uid_cv[model].mean() if len(uid_cv) > 0 else None
                    tt_mae = uid_traintest[model].iloc[0] if len(uid_traintest) > 0 and model in uid_traintest.columns else None
                    
                    result_row[f'{model}_cv_mae'] = cv_mae
                    result_row[f'{model}_tt_mae'] = tt_mae
                
                all_results.append(result_row)
                
                # One row per unique_id per configuration with MEAN CV MAE and Train-Test MAE
                error_row = {
                    'config_id': i + 1,
                    'unique_id': uid,
                    'population_scenario': population_scenario,
                    **params,
                    'n_cv_windows': cv_total,
                }
                
                # Add mean CV MAE and Train-Test MAE for each model
                for model in model_cols:
                    # Mean MAE across all CV windows
                    cv_mean_mae = uid_cv[model].mean() if len(uid_cv) > 0 else None
                    error_row[f'{model}_cv_mean_mae'] = cv_mean_mae
                    
                    # Train-Test MAE
                    tt_mae = uid_traintest[model].iloc[0] if len(uid_traintest) > 0 and model in uid_traintest.columns else None
                    error_row[f'{model}_tt_mae'] = tt_mae
                
                all_errors.append(error_row)
            
            print(f"   ✅ Configuration {i+1} completed successfully")
            
        except Exception as e:
            print(f"   ❌ Error in configuration {i+1}: {str(e)}")
            import traceback
            traceback.print_exc()
            continue
    
    # Convert to DataFrames
    results_df = pd.DataFrame(all_results)
    errors_df = pd.DataFrame(all_errors)
    
    print(f"\n✅ Sensitivity analysis complete!")
    print(f"\nTotal configurations tested: {results_df['config_id'].nunique()}")
    print(f"\nTotal unique_ids analyzed: {results_df['unique_id'].nunique()}")
    
    print("\nPhase 3: Identifying best models per horizon...")
    
    recommendations_per_horizon = {}
    
    for horizon in production_horizons:
        print(f"\nAnalyzing horizon h={horizon}...")
        
        horizon_results = results_df[results_df['h'] == horizon].copy()
        
        if len(horizon_results) == 0:
            print(f"   ⚠️ No results for h={horizon}")
            continue
        
        horizon_recommendations = []
        
        for uid in horizon_results['unique_id'].unique():
            uid_data = horizon_results[horizon_results['unique_id'] == uid]
            
            # Most frequent CV best model
            cv_mode = uid_data['cv_best_model'].mode()
            cv_best = cv_mode.iloc[0] if len(cv_mode) > 0 else None
            cv_freq = (uid_data['cv_best_model'] == cv_best).sum() / len(uid_data)
            
            # Most frequent train-test best model
            tt_mode = uid_data['traintest_best_model'].mode()
            tt_best = tt_mode.iloc[0] if len(tt_mode) > 0 else None
            tt_freq = (uid_data['traintest_best_model'] == tt_best).sum() / len(uid_data)
            
            # Average consistency
            avg_consistency = uid_data['cv_consistency'].mean()
            
            # Determine recommendation
            if cv_best == tt_best and cv_freq >= 0.7 and avg_consistency >= 0.7:
                confidence = 'High'
                recommended_model = cv_best
                reason = "CV and Train-Test agree with high frequency and consistency"
            elif cv_best == tt_best and cv_freq >= 0.5:
                confidence = 'Medium-High'
                recommended_model = cv_best
                reason = "CV and Train-Test agree with moderate frequency"
            elif cv_freq >= 0.6:
                confidence = 'Medium'
                recommended_model = cv_best
                reason = f"CV favors {cv_best} ({cv_freq:.0%})"
            elif tt_freq >= 0.6:
                confidence = 'Medium'
                recommended_model = tt_best
                reason = f"Train-Test favors {tt_best} ({tt_freq:.0%})"
            else:
                confidence = 'Low'
                # Use lowest average CV MAE
                mae_cols = [col for col in uid_data.columns if col.endswith('_cv_mae')]
                if mae_cols:
                    avg_maes = uid_data[mae_cols].mean()
                    recommended_model = avg_maes.idxmin().replace('_cv_mae', '')
                else:
                    recommended_model = cv_best
                reason = "No clear winner - using lowest average CV MAE"
            
            # Get average MAE
            rec_mae_col = f'{recommended_model}_cv_mae'
            avg_mae = uid_data[rec_mae_col].mean() if rec_mae_col in uid_data.columns else None
            
            recommendation = {
                'unique_id': uid,
                'horizon': horizon,
                'recommended_model': recommended_model,
                'confidence': confidence,
                'reason': reason,
                'cv_best_model': cv_best,
                'cv_frequency': cv_freq,
                'cv_consistency': avg_consistency,
                'traintest_best_model': tt_best,
                'avg_cv_mae': avg_mae,
                'population_scenario': population_scenario
            }
            
            horizon_recommendations.append(recommendation)
            
            mae_display = f"{avg_mae:,.0f}" if pd.notna(avg_mae) else "N/A"
            print(f"   {uid}: {recommended_model} (confidence: {confidence}, MAE: {mae_display})")
        
        recommendations_per_horizon[horizon] = pd.DataFrame(horizon_recommendations)
    
    print("\nPhase 4: Generating production forecasts...")
    
    production_forecasts = {}
    
    for horizon in production_horizons:
        if horizon not in recommendations_per_horizon:
            continue
        
        print(f"\nGenerating forecasts for h={horizon}...")
        
        recommendations = recommendations_per_horizon[horizon]
        
        # Configuration for production
        config_prod = ForecastConfig(
            h=horizon,
            season_length=4,
            confidence_level=95,
            n_jobs=-1
        )
        
        # Train all models on full historical data
        models = get_models_exog(config_prod, use_exog=True)
        
        sf = StatsForecast(
            models=models,
            freq='QS',
            n_jobs=-1,
            fallback_model=SeasonalNaive(season_length=4)
        )
        
        # Prepare data with population
        pop_historical = pop_df[pop_df['ds'] <= df['ds'].max()].copy()
        df_train = df.merge(pop_historical, on=['unique_id', 'ds'], how='left')
        
        # Fit on historical data
        sf.fit(df=df_train[['unique_id', 'ds', 'y', 'population']])
        
        # Prepare future population
        last_date = df['ds'].max()
        future_dates = pd.date_range(
            start=last_date + pd.DateOffset(months=3),
            periods=horizon,
            freq='QS'
        )
        
        future_exog_list = []
        for uid in df['unique_id'].unique():
            for date in future_dates:
                pop_val = pop_df[
                    (pop_df['unique_id'] == uid) & 
                    (pop_df['ds'] == date)
                ]['population'].values
                
                if len(pop_val) > 0:
                    future_exog_list.append({
                        'unique_id': uid,
                        'ds': date,
                        'population': pop_val[0]
                    })
        
        future_exog_df = pd.DataFrame(future_exog_list)
        
        # Generate forecasts
        forecasts_df = sf.predict(h=horizon, X_df=future_exog_df)
        
        # Create best model forecasts
        best_forecasts = []
        
        for uid in df['unique_id'].unique():
            uid_forecasts = forecasts_df.reset_index()
            uid_forecasts = uid_forecasts[uid_forecasts['unique_id'] == uid]
            
            uid_rec = recommendations[recommendations['unique_id'] == uid]
            if len(uid_rec) == 0:
                print(f"   ⚠️ No recommendation for {uid}, skipping")
                continue
            
            best_model = uid_rec['recommended_model'].iloc[0]
            
            for _, row in uid_forecasts.iterrows():
                best_row = {
                    'unique_id': uid,
                    'ds': row['ds'],
                    'recommended_model': best_model,
                    'forecast': row.get(best_model, np.nan),
                    'forecast_lo_95': row.get(f'{best_model}-lo-95', np.nan),
                    'forecast_hi_95': row.get(f'{best_model}-hi-95', np.nan),
                    'population_scenario': population_scenario
                }
                best_forecasts.append(best_row)
        
        best_forecasts_df = pd.DataFrame(best_forecasts)
        
        production_forecasts[horizon] = {
            'all_models': forecasts_df,
            'best_model': best_forecasts_df,
            'recommendations': recommendations
        }
        
        print(f"   ✅ Generated {len(best_forecasts_df)} forecast periods")
        print(f"   ✅ Models used: {recommendations['recommended_model'].value_counts().to_dict()}")
    
    print("\nPhase 5: Saving results...")
    
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        
        # Save comprehensive results
        sensitivity_file = os.path.join(
            save_path, 
            f'{target_name.lower().replace(" ", "_")}_sensitivity_{population_scenario}.xlsx'
        )
        
        with pd.ExcelWriter(sensitivity_file, engine='openpyxl') as writer:
            # Sheet 1: All configurations summary
            results_df.to_excel(writer, sheet_name='All_Configurations', index=False)
            
            # Sheet 2: Simplified errors - Mean CV MAE and Train-Test MAE per config
            errors_df.to_excel(writer, sheet_name='Model_Errors', index=False)
            
            # Sheet 3: Recommendations
            rec_frames = list(recommendations_per_horizon.values())
            if rec_frames:
                all_recs = pd.concat(rec_frames, ignore_index=True)
                all_recs.to_excel(writer, sheet_name='Recommendations', index=False)
            
            # Sheet 4: Configuration summary statistics
            config_summary = results_df.groupby('config_id').agg({
                'cv_traintest_agree': 'mean',
                'cv_consistency': 'mean'
            }).reset_index()
            config_summary.columns = ['config_id', 'avg_cv_tt_agreement', 'avg_cv_consistency']
            
            # Add parameter details
            param_details = results_df.groupby('config_id')[param_names].first().reset_index()
            config_summary = config_summary.merge(param_details, on='config_id')
            config_summary.to_excel(writer, sheet_name='Config_Summary', index=False)
        
        print(f"✅ Sensitivity analysis saved: {sensitivity_file}")
        
        # Save production forecasts
        for horizon, forecasts in production_forecasts.items():
            horizon_file = os.path.join(
                save_path,
                f'{target_name.lower().replace(" ", "_")}_production_h{horizon}_{population_scenario}.xlsx'
            )
            
            with pd.ExcelWriter(horizon_file, engine='openpyxl') as writer:
                forecasts['all_models'].reset_index().to_excel(
                    writer, sheet_name='All_Models', index=False
                )
                forecasts['best_model'].to_excel(
                    writer, sheet_name='Best_Model_Forecast', index=False
                )
                forecasts['recommendations'].to_excel(
                    writer, sheet_name='Model_Selection', index=False
                )
            
            print(f"✅ Production forecasts (h={horizon}) saved: {horizon_file}")
    else:
        print("   ⚠️ No save_path provided - skipping file export")
    
    print("\n" + "=" * 80)
    print("✅ SENSITIVITY ANALYSIS COMPLETE!")
    print("=" * 80)
    
    return {
        'sensitivity_results': results_df,
        'model_errors': errors_df,
        'production_forecasts': production_forecasts,
        'recommendations_per_horizon': recommendations_per_horizon
    }

In [ ]:
filepath_withpop = rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\Prebuilt_panels\P1_withpop.csv"
save_path = rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\Forecast\TMF\\"

In [ ]:
# Configure
config = ForecastConfig(
    h=8,
    season_length=4,
    n_windows=3,
    train_size=None,
    test_size=8,  #Setting it to the same h
    n_samples=4,
    confidence_level=95,
    models_to_plot=['Naive', 'SARIMAX']
)

# Run forecasting for Units Reimbursed with all three population scenarios
results_units = forecast_with_population_scenarios(
    filepath=filepath_withpop,
    target_col='Units Reimbursed',
    states=['IN'],  # Or None for all states
    config=config,
    horizons=[8],
    save_path=save_path
)

# Run forecasting for Number of Prescriptions
results_prescriptions = forecast_with_population_scenarios(
    filepath=filepath_withpop,
    target_col='Number of Prescriptions',
    states=['IN'],
    config=config,
    horizons=[8],
    save_path=save_path
)

In [ ]:
# Example usage
results = sensitivity_analysis_with_exog(
    filepath=filepath_withpop,
    target_col='Number of Prescriptions',
    states=['IN'],
    param_grid={
        'h': [4, 8],
        'train_size': [None], #Training all the dataset for Cross-Validation, otherwise the n_windows will be too small
        'n_windows': [2, 3],
        'step_size': [4],
    },
    production_horizons=[4, 8],
    population_scenario='high_95',  # 'point' or 'low_95' or 'high_95'
    save_path=save_path
)